In [55]:
!pip install openai-whisper python-docx --quiet
!sudo apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [56]:
import whisper
import re
from docx import Document
from datetime import datetime
from google.colab import files

In [57]:
import re

def split_letters_numbers(text):
    return re.sub(r'([a-zA-Z]{2,})(\d+)', r'\1 \2', text)

In [58]:
def normalize_transcription(text):
    corecturi = {
        "a orta": "aorta",
        "aorta inel": "aorta la inel",
        "si nosuri": "sinusuri",
        "acendentă": "ascendentă",
        "aste": "AS",
        "ase": "AS",
        "veste": "VS",
        "vse": "VS",
        "rede posterior": "peretele posterior",
        "perete posterior": "peretele posterior",
        "ijecție": "ejecție",
        "șaezeci": "șaizeci",
        "virglă": ",",
        "ve max": "Vmax",
        "pul monară": "pulmonară",
        "valva ortică": "valva aortică",
        "douăși": "24",
        "pru cinci": "+5",
        "milimentru": "milimetru",
        "injecție": "ejecție"
    }
    for gresit, corect in corecturi.items():
        text = text.replace(gresit, corect)
    text = re.sub(r"\bvs\s+(\d+)\s+pe\s+(\d+)", r"vs \1/\2", text, flags=re.IGNORECASE)

    return text

def convert_romanian_numbers(text):
    numere = {
        "unu": "1", "una": "1", "unul": "1",
        "doi": "2", "două": "2",
        "trei": "3", "patru": "4",
        "cinci": "5", "șase": "6", "șapte": "7",
        "opt": "8", "nouă": "9", "zece": "10",
        "unsprezece": "11", "doisprezece": "12", "douăsprezece": "12",
        "treisprezece": "13", "paisprezece": "14",
        "cincisprezece": "15", "șaisprezece": "16", "șaptesprezece": "17",
        "optsprezece": "18", "nouăsprezece": "19", "douăzeci": "20",
        "șaizeci": "60", "șaptezeci": "70"
    }
    words = text.lower().split()
    return " ".join([numere.get(w, w) for w in words])

def fix_virgula_valori(text):
    return re.sub(r"(\d+)\s*(virglă|,)\s*(\d+)", r"\1.\3", text)

def cleanup_transcription(text):
    text = re.sub(r'(\d)([a-zăîâșț])', r'\1 \2', text)
    return text


In [59]:
model = whisper.load_model("small")

def transcribe_and_clean(file_path):
    result = model.transcribe(file_path, language="ro")
    raw = result["text"].lower()
    text = normalize_transcription(raw)
    text = convert_romanian_numbers(text)
    text = split_letters_numbers(text)
    text = cleanup_transcription(text)
    return text


In [ ]:
def extract_medical_values(text):
    values = {
        "Ao_inel": re.search(r"aorta la inel\s*[:\-]?\s*(\d+)", text, re.IGNORECASE),
        "Ao_sinusuri": re.search(r"aorta\s+(?:la\s+)?sinusur[i]?[^\d]{0,5}(\d+)", text, re.IGNORECASE),
        "Ao_ascendenta": re.search(r"aorta ascendent[ăa]?\s*[:\-]?\s*(\d+)", text, re.IGNORECASE),
        "AS": re.search(r"\bas\s*[:\-]?\s*(\d+)", text, re.IGNORECASE),
        "VD": re.search(r"\bvd\s*[:\-]?\s*(\d+)", text, re.IGNORECASE),
        "SIV": re.search(r"\bsiv\s*[:\-]?\s*(\d+)", text, re.IGNORECASE),
        "VS": re.search(r"\bvs\s*[:\-]?\s*(\d+[/]?\d*)", text, re.IGNORECASE),
        "PP": (
            re.search(r"\bpp\s*[:\-]?\s*(\d+)", text, re.IGNORECASE)
            or re.search(r"peretele posterior\s*[:\-]?\s*(\d+)", text, re.IGNORECASE)
        ),
        "FE": re.search(r"(fracție de ejecție|FE)[^\d]{0,10}(\d+[.,]?\d*)", text, re.IGNORECASE),
        "Vmax_aorta": re.search(r"valva aortic[ăa]?\s*vmax\s*[:\-]?\s*(\d+[.,]?\d*)", text, re.IGNORECASE),
        "Vmax_pulm": re.search(r"valva pulmonar[ăa].*?vmax\s*[:\-]?\s*(\d+[.,]?\d*)", text, re.IGNORECASE),
        "Vmax_tricuspidian": re.search(r"valva tricuspidian[ăa].*?vmax\s*[:\-]?\s*(\d+[.,]?\d*)", text, re.IGNORECASE),
        "PSVD": re.search(r"\bpsvd\s*[:\-]?\s*(\d+)[^\d+]*(?:\+)?(\d+)?", text, re.IGNORECASE),
    }

    results = {}
    for k, v in values.items():
        if isinstance(v, re.Match):
            if k == "PSVD" and v.groups()[1]:
                results[k] = f"{v.group(1)}+{v.group(2)}"
            elif k == "FE":
                results[k] = v.group(2)
            else:
                results[k] = v.group(1)
        else:
            results[k] = "nedefinit"

    return results


In [ ]:
def save_values_to_word(values_dict, filename="fisa_extrasa.docx"):
    doc = Document()
    doc.add_heading("Date extrase din voce - Ecografie", 0)
    doc.add_paragraph(f"Generat: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    table = doc.add_table(rows=1, cols=2)
    table.style = 'Table Grid'
    table.autofit = True

    hdr = table.rows[0].cells
    hdr[0].text = "Parametru"
    hdr[1].text = "Valoare"

    for k, v in values_dict.items():
        row = table.add_row().cells
        row[0].text = k
        row[1].text = v

    doc.save(filename)
    return filename


In [ ]:
uploaded = files.upload()
file_path = next(iter(uploaded))

text = transcribe_and_clean(file_path)
print(" Transcriere curățată:\n", text)

valori = extract_medical_values(text)
print("\n Valori extrase:")
for k, v in valori.items():
    print(f"{k}: {v}")

word_file = save_values_to_word(valori)
files.download(word_file)


Saving test3.ogg to test3 (6).ogg


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Transcriere curățată:
 aorta la inel 8, aorta sinusur 12, aorta ascendenta 10, as 13, vd 6, siv 3, vs 20/12, peretele posterior 4, fracție de ejecție 60%, tap 13, bar 6, bar 6, valva aortic vmax 1,2, valva pulmonară vmax 1,1, valva tricuspidiană vmax 2,5, psvd 24,5, arca aortic vmax 1,3, alte detalii, dacă ar merge băgat, pca mic restrictiv cu diametru de aproximativ un milimetru.

 Valori extrase:
Ao_inel: 8
Ao_sinusuri: 12
Ao_ascendenta: 10
AS: 13
VD: 6
SIV: 3
VS: 20/12
PP: 4
FE: 60
Vmax_aorta: 1,2
Vmax_pulm: 1,1
Vmax_tricuspidian: 2,5
PSVD: 24+5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>